In [1]:
import pandas as pd
import numpy as np
import os

file_path = r"C:\Users\simonk03\Downloads\Mid Project\Final\Provision2024.xlsx"  
if os.path.exists(file_path):  # Check if the file exists
    excel_data = pd.ExcelFile(file_path)  # Load the Excel file
    actual_potential = excel_data.parse("Actual&Potential")  # Load 'Actual&Potential' sheet
    loss_tree = excel_data.parse("Loss Tree")  # Load 'Loss Tree' sheet
    print("File loaded successfully!")  # Confirm successful loading
else:
    print(f"File not found at {file_path}. Please check the path and try again.")  #if file is not found|


File loaded successfully!


In [2]:
actual_potential.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203 entries, 0 to 1202
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Segment           10 non-null     object 
 1   Plant             1203 non-null   object 
 2   Product           1182 non-null   object 
 3   Quantity          918 non-null    object 
 4   Risk Level        1193 non-null   object 
 5   Segement          1193 non-null   object 
 6   Code              1190 non-null   object 
 7   UOM               1193 non-null   object 
 8   Status            1193 non-null   object 
 9   Classification    1193 non-null   object 
 10  Level 1           1187 non-null   object 
 11  Level 2           1187 non-null   object 
 12  Level 3           1187 non-null   object 
 13  Scrapping Status  1193 non-null   object 
 14  Scrapping Month   1193 non-null   float64
 15  Value in EGP      1192 non-null   float64
 16  Date              10 non-null     object 


In [3]:
loss_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 28 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Level 1                                    26 non-null     object 
 1   Product_Damaged                            10 non-null     object 
 2   Product_Expired                            8 non-null      object 
 3   Product_Not_Meeting_with_Specs_to_Be_Sold  8 non-null      object 
 4   Missing_Product                            9 non-null      object 
 5   Cancelled_NPI                              5 non-null      object 
 6   Quality_Issue                              10 non-null     object 
 7   Delisted_SKU                               15 non-null     object 
 8   Postponed_NPI                              7 non-null      object 
 9   Unnamed: 9                                 1 non-null      object 
 10  Unnamed: 10                 

In [4]:
# Step 1: Handle Missing Values in "Actual&Potential"
actual_potential['Product'].fillna('Unknown', inplace=True) # Fill missing 'Product' with 'Unknown'
actual_potential['Risk Level'].fillna('Unknown', inplace=True) # Fill missing 'Risk Level' with 'Unknown'
actual_potential['Quantity'] = pd.to_numeric(actual_potential['Quantity'], errors='coerce') # Convert 'Quantity' to numeric
actual_potential['Quantity'].fillna(actual_potential['Quantity'].mean(), inplace=True) # Fill NaN 'Quantity' with the mean value
print("Step 1: Missing values handled in 'Actual&Potential'.") 


Step 1: Missing values handled in 'Actual&Potential'.


In [5]:
# Step 2: Handle Missing Values in "Loss Tree"

# Check if the 'Category' column exists in the DataFrame
if 'Category' in loss_tree.columns:
    # Convert 'Category' to string type and fill missing values with 'Unknown'
    loss_tree['Category'] = loss_tree['Category'].astype(str)
    loss_tree['Category'].fillna('Unknown', inplace=True)
else:
    # Log a message if 'Category' column is not found
    print("'Category' column not found in 'Loss Tree'.")

# Check if the 'Sub-Category' column exists in the DataFrame
if 'Sub-Category' in loss_tree.columns:
    # Convert 'Sub-Category' to string type and fill missing values with 'Unknown'
    loss_tree['Sub-Category'] = loss_tree['Sub-Category'].astype(str)
    loss_tree['Sub-Category'].fillna('Unknown', inplace=True)
else:
    # Log a message if 'Sub-Category' column is not found
    print("'Sub-Category' column not found in 'Loss Tree'.")

# Log the completion of missing value handling
print("Missing values handled in 'Loss Tree'.")


'Sub-Category' column not found in 'Loss Tree'.
Missing values handled in 'Loss Tree'.


In [6]:
# Step 2: Handle Duplicates in "Actual&Potential"
# Removing rows that have the same 'Product', 'Risk Level', and 'Quantity'
# We'll keep the first occurrence and drop others
actual_potential.drop_duplicates(subset=['Product', 'Risk Level', 'Quantity'], keep='first', inplace=True)
print("Duplicates removed in 'Actual&Potential'.")


Duplicates removed in 'Actual&Potential'.


In [7]:
# Step 3: Handle Inconsistent Data Types in "Actual&Potential"
# Convert 'Quantity' column to numeric, invalid values will be set as NaN
actual_potential['Quantity'] = pd.to_numeric(actual_potential['Quantity'], errors='coerce')
# Convert 'Product' to string type (even if it's already a string, this ensures consistency)
actual_potential['Product'] = actual_potential['Product'].astype(str)
print("Data types fixed in 'Actual&Potential'.")


Data types fixed in 'Actual&Potential'.


In [8]:
# Step 4: Handle Misplaced Rows/Irrelevant Data in "Actual&Potential"
# Remove rows where 'Segment' is '---' and 'Plant' is '###'
actual_potential = actual_potential[actual_potential['Segment'] != '---']
actual_potential = actual_potential[actual_potential['Plant'] != '###']
print("Misplaced rows removed in 'Actual&Potential'.")

Misplaced rows removed in 'Actual&Potential'.


In [9]:
# Step 5: Handle Outliers in "Actual&Potential"
# Replace extreme values (above 1000) with NaN and then fill with the column mean
actual_potential['Quantity'] = actual_potential['Quantity'].apply(lambda x: x if x < 1000 else np.nan)
actual_potential['Quantity'].fillna(actual_potential['Quantity'].mean(), inplace=True)
print("Outliers handled in 'Actual&Potential'.")

Outliers handled in 'Actual&Potential'.


In [10]:
# Step 6: Handle Incorrect Date Formats in "Actual&Potential"
# Convert 'Date' column to datetime, replacing invalid formats with NaT
actual_potential['Date'] = pd.to_datetime(actual_potential['Date'], errors='coerce')

# Fill missing date values with a default date
actual_potential['Date'].fillna('2024-01-01', inplace=True)
print("Date formats corrected in 'Actual&Potential'.")

Date formats corrected in 'Actual&Potential'.


In [11]:
# Step 7: Handle Invalid Column Headers in "Actual&Potential" & Remove duplicated column names
actual_potential = actual_potential.loc[:, ~actual_potential.columns.duplicated()]
print("Invalid column headers resolved in 'Actual&Potential'.")


Invalid column headers resolved in 'Actual&Potential'.


In [12]:
# Step 8: Handle Loss Tree Specific Errors & Handle Missing Values in "Loss Tree"
loss_tree['Category'] = loss_tree.get('Category', pd.Series('Unknown', index=loss_tree.index)).fillna('Unknown')
loss_tree['Sub-Category'] = loss_tree.get('Sub-Category', pd.Series('Unknown', index=loss_tree.index)).fillna('Unknown')
print("Missing values handled in 'Loss Tree'.")


Missing values handled in 'Loss Tree'.


In [13]:
# Step 9: Save the cleaned data to a new Excel file
output_path = r'C:\Users\simonk03\Documents\cleaned_scrapping_provision_from_modified1.xlsx'  # Changed path
try:
    with pd.ExcelWriter(output_path) as writer:
        actual_potential.to_excel(writer, index=False, sheet_name="Actual&Potential")
        loss_tree.to_excel(writer, index=False, sheet_name="Loss Tree")
    print(f"Data cleaned and saved to '{output_path}'.")
except Exception as e:
    print(f"Error saving the file: {e}")


Data cleaned and saved to 'C:\Users\simonk03\Documents\cleaned_scrapping_provision_from_modified1.xlsx'.


In [14]:
# Convert 'Scrapping Month' to numeric, remove out-of-bounds values, and convert to datetime
actual_potential['Scrapping Month'] = pd.to_datetime(
    pd.to_numeric(actual_potential['Scrapping Month'], errors='coerce').clip(lower=1), 
    origin='unix', unit='D', errors='coerce').dt.strftime('%B %Y')

print("Scrapping Month format changed to 'Month Year'.")


Scrapping Month format changed to 'Month Year'.


In [15]:
# Step 9: Save the cleaned data to a new Excel file
output_path = r'C:\Users\simonk03\Documents\cleaned_scrapping_provision_from_modified1.xlsx'  # Changed path
try:
    with pd.ExcelWriter(output_path) as writer:
        actual_potential.to_excel(writer, index=False, sheet_name="Actual&Potential")
        loss_tree.to_excel(writer, index=False, sheet_name="Loss Tree")
    print(f"Data cleaned and saved to '{output_path}'.")
except Exception as e:
    print(f"Error saving the file: {e}")


Data cleaned and saved to 'C:\Users\simonk03\Documents\cleaned_scrapping_provision_from_modified1.xlsx'.


In [1]:
# Save the data as a CSV file
csv_file_path = r'C:\Users\simonk03\OneDrive - Heineken International\Data siense\Master Data sciense\PowerBI Projects\Mid project\Finaalsss\Mid project Scrapping\cleaned_scrapping_provision_from_modified1.csv'
actual_potential.to_csv(csv_file_path, index=False)

print(f"Data successfully saved to {csv_file_path}")

NameError: name 'actual_potential' is not defined